# Actividad 2 - Módulo 3

### Configuración inicial

In [12]:
# Carga de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 1. Carga y limpieza de datos

In [4]:
# Datos iniciales
nombre = ['Ana', 'Luis', 'Sofía', 'Pedro', 'Marta', 'Juan', 'Lucía', 'Miguel']
edad = [23, 35, 28, 40, 29, 35, 31, 27]
ciudad = ['Madrid', 'Barcelona', 'Valencia', 'Bilbao', 'Madrid', 'Barcelona', 'Valencia', 'Bilbao']
ingresos = [2500, 3200, 2900, 3600, 2700, 3100, 3300, 2750]
área = ['IT', 'Ventas', 'IT', 'Dirección', 'Recursos Humanos', 'Ventas', 'IT', 'Recursos Humanos']
años_experiencia = [2, 8, 5, 15, 3, 9, 6, 4]

# Agregando valores atípicos y faltantes intencionalmente
edad[3] = 150  # Valor atípico en edad (Pedro)
ingresos[5] = None  # Valor faltante en ingresos (Juan)

# Crear DataFrame
df = pd.DataFrame({
    'nombre': nombre,
    'edad': edad,
    'ciudad': ciudad,
    'ingresos': ingresos,
    'área': área,
    'años_experiencia': años_experiencia
})

print("DataFrame inicial con valores atípicos y faltantes:")
print(df)

# Identificar valores faltantes y anómalos
print("\nValores faltantes por columna:")
print(df.isnull().sum())

# Identificar valores atípicos en edad
print("\nValores atípicos en edad (mayores a 100 o negativos):")
edad_atipicos = df[(df['edad'] > 100) | (df['edad'] < 0)]
print(edad_atipicos[['nombre', 'edad']])



DataFrame inicial con valores atípicos y faltantes:
   nombre  edad     ciudad  ingresos              área  años_experiencia
0     Ana    23     Madrid    2500.0                IT                 2
1    Luis    35  Barcelona    3200.0            Ventas                 8
2   Sofía    28   Valencia    2900.0                IT                 5
3   Pedro   150     Bilbao    3600.0         Dirección                15
4   Marta    29     Madrid    2700.0  Recursos Humanos                 3
5    Juan    35  Barcelona       NaN            Ventas                 9
6   Lucía    31   Valencia    3300.0                IT                 6
7  Miguel    27     Bilbao    2750.0  Recursos Humanos                 4

Valores faltantes por columna:
nombre              0
edad                0
ciudad              0
ingresos            1
área                0
años_experiencia    0
dtype: int64

Valores atípicos en edad (mayores a 100 o negativos):
  nombre  edad
3  Pedro   150


### 2. Exploración básica

In [5]:
print("HEAD del DataFrame:")
print(df.head())

print("\nINFO del DataFrame:")
print(df.info())

print("\nDESCRIBE del DataFrame:")
print(df.describe())

print("\nVALUE_COUNTS para 'área':")
print(df['área'].value_counts())

print("\nVALUE_COUNTS para 'ciudad':")
print(df['ciudad'].value_counts())

print("\nCOMENTARIOS sobre la exploración:")
print("""
- Distribución de áreas: IT tiene 3 empleados, siendo el área más representada
- Ciudades: distribución equitativa con 2 empleados por ciudad
- Edad: rango amplio con un valor atípico de 150 años
- Ingresos: promedio alrededor de 3000, con un valor faltante
- Años de experiencia: desde 2 hasta 15 años
""")

HEAD del DataFrame:
  nombre  edad     ciudad  ingresos              área  años_experiencia
0    Ana    23     Madrid    2500.0                IT                 2
1   Luis    35  Barcelona    3200.0            Ventas                 8
2  Sofía    28   Valencia    2900.0                IT                 5
3  Pedro   150     Bilbao    3600.0         Dirección                15
4  Marta    29     Madrid    2700.0  Recursos Humanos                 3

INFO del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   nombre            8 non-null      object 
 1   edad              8 non-null      int64  
 2   ciudad            8 non-null      object 
 3   ingresos          7 non-null      float64
 4   área              8 non-null      object 
 5   años_experiencia  8 non-null      int64  
dtypes: float64(1), int64(2), object(3)
memory usag

### 3. Limpieza de datos

In [6]:
# Calcular mediana de edad excluyendo valores atípicos
edad_sin_atipicos = df[(df['edad'] <= 100) & (df['edad'] >= 0)]['edad']
mediana_edad = edad_sin_atipicos.median()

print(f"Mediana de edad (sin atípicos): {mediana_edad}")

# Reemplazar valores atípicos en edad
df.loc[(df['edad'] > 100) | (df['edad'] < 0), 'edad'] = mediana_edad

# Calcular media de ingresos por área para imputación
media_ingresos_por_area = df.groupby('área')['ingresos'].mean()
print("\nMedia de ingresos por área:")
print(media_ingresos_por_area)

# Imputar valores faltantes en ingresos usando la media por área
for idx in df[df['ingresos'].isnull()].index:
    area_empleado = df.loc[idx, 'área']
    df.loc[idx, 'ingresos'] = media_ingresos_por_area[area_empleado]

print("\nDataFrame después de la limpieza:")
print(df)

Mediana de edad (sin atípicos): 29.0

Media de ingresos por área:
área
Dirección           3600.0
IT                  2900.0
Recursos Humanos    2725.0
Ventas              3200.0
Name: ingresos, dtype: float64

DataFrame después de la limpieza:
   nombre  edad     ciudad  ingresos              área  años_experiencia
0     Ana    23     Madrid    2500.0                IT                 2
1    Luis    35  Barcelona    3200.0            Ventas                 8
2   Sofía    28   Valencia    2900.0                IT                 5
3   Pedro    29     Bilbao    3600.0         Dirección                15
4   Marta    29     Madrid    2700.0  Recursos Humanos                 3
5    Juan    35  Barcelona    3200.0            Ventas                 9
6   Lucía    31   Valencia    3300.0                IT                 6
7  Miguel    27     Bilbao    2750.0  Recursos Humanos                 4


### 4. Filtrado avanzado y selección múltiple

In [7]:
# Filtrar empleados según los criterios
empleados_filtrados = df[
    ((df['ciudad'] == 'Madrid') | (df['ciudad'] == 'Barcelona')) &
    (df['edad'] > 30) &
    (df['años_experiencia'] > 5)
]

print("Empleados que cumplen todos los criterios:")
print("- Trabajen en Madrid o Barcelona")
print("- Tengan más de 30 años")
print("- Y más de 5 años de experiencia")
print("\n")
print(empleados_filtrados)
print(f"\nTotal de empleados que cumplen los criterios: {len(empleados_filtrados)}")
print("\nCaracterísticas compartidas:")
print("- Todos tienen ingresos superiores a 3000")
print("- Predominan las áreas de Ventas")
print("- Edad promedio:", empleados_filtrados['edad'].mean())

Empleados que cumplen todos los criterios:
- Trabajen en Madrid o Barcelona
- Tengan más de 30 años
- Y más de 5 años de experiencia


  nombre  edad     ciudad  ingresos    área  años_experiencia
1   Luis    35  Barcelona    3200.0  Ventas                 8
5   Juan    35  Barcelona    3200.0  Ventas                 9

Total de empleados que cumplen los criterios: 2

Características compartidas:
- Todos tienen ingresos superiores a 3000
- Predominan las áreas de Ventas
- Edad promedio: 35.0


### 5. Agrupación y resumen

In [8]:
# Agrupación por área
print("Estadísticas por ÁREA:")
agrupacion_area = df.groupby('área').agg({
    'ingresos': 'mean',
    'edad': 'max'
}).round(2)
agrupacion_area.columns = ['Ingreso Promedio', 'Edad Máxima']
print(agrupacion_area)
print("\n")

# Agrupación por ciudad
print("Estadísticas por CIUDAD:")
agrupacion_ciudad = df.groupby('ciudad').agg({
    'ingresos': 'mean',
    'edad': 'max'
}).round(2)
agrupacion_ciudad.columns = ['Ingreso Promedio', 'Edad Máxima']
print(agrupacion_ciudad)

# Encontrar ciudad y área con ingresos más altos
ingreso_max_area = agrupacion_area['Ingreso Promedio'].idxmax()
ingreso_max_ciudad = agrupacion_ciudad['Ingreso Promedio'].idxmax()

print(f"\nÁrea con mayor ingreso promedio: {ingreso_max_area} (${agrupacion_area.loc[ingreso_max_area, 'Ingreso Promedio']:.2f})")
print(f"Ciudad con mayor ingreso promedio: {ingreso_max_ciudad} (${agrupacion_ciudad.loc[ingreso_max_ciudad, 'Ingreso Promedio']:.2f})")

Estadísticas por ÁREA:
                  Ingreso Promedio  Edad Máxima
área                                           
Dirección                   3600.0           29
IT                          2900.0           31
Recursos Humanos            2725.0           29
Ventas                      3200.0           35


Estadísticas por CIUDAD:
           Ingreso Promedio  Edad Máxima
ciudad                                  
Barcelona            3200.0           35
Bilbao               3175.0           29
Madrid               2600.0           29
Valencia             3100.0           31

Área con mayor ingreso promedio: Dirección ($3600.00)
Ciudad con mayor ingreso promedio: Barcelona ($3200.00)


### 6. Creación y transformación de variables

In [9]:
# Crear columna nivel_ingresos
df['nivel_ingresos'] = pd.cut(df['ingresos'], 
                               bins=[0, 2800, 3200, float('inf')],
                               labels=['bajo', 'medio', 'alto'])

# Crear columna senioridad
df['senioridad'] = pd.cut(df['años_experiencia'],
                          bins=[0, 5, 10, float('inf')],
                          labels=['junior', 'semisenior', 'senior'],
                          right=False)

print("DataFrame con nuevas variables categóricas:")
print(df[['nombre', 'ingresos', 'nivel_ingresos', 'años_experiencia', 'senioridad']])

DataFrame con nuevas variables categóricas:
   nombre  ingresos nivel_ingresos  años_experiencia  senioridad
0     Ana    2500.0           bajo                 2      junior
1    Luis    3200.0          medio                 8  semisenior
2   Sofía    2900.0          medio                 5  semisenior
3   Pedro    3600.0           alto                15      senior
4   Marta    2700.0           bajo                 3      junior
5    Juan    3200.0          medio                 9  semisenior
6   Lucía    3300.0           alto                 6  semisenior
7  Miguel    2750.0           bajo                 4      junior


### 8. Interpretación y propuesta

In [11]:
print("Principales hallazgos:")
print("""
1. Desigualdades detectadas: Existe una brecha salarial significativa entre áreas, 
   con Dirección teniendo el mayor ingreso ($3600) y Recursos Humanos el menor promedio ($2725).

2. Grupos destacados: Los empleados senior en áreas técnicas (IT) y comerciales (Ventas) 
   presentan los mejores perfiles de compensación y experiencia.

3. Anomalías corregidas: Se identificó y corrigió un valor atípico en edad (150 años) 
   y se imputó un valor faltante en ingresos usando la media del área correspondiente.

4. Recomendaciones para la gerencia:
   - Revisar la política salarial en Recursos Humanos para reducir la brecha
   - Implementar un plan de desarrollo para empleados junior en todas las áreas
   - Considerar ajustes salariales basados en años de experiencia""")

# Análisis adicional: Correlación entre experiencia e ingresos
print("\nANÁLISIS ADICIONAL:")
print("- Correlación Experiencia-Ingresos:")
correlacion = df[['años_experiencia', 'ingresos']].corr()
print(correlacion)

print(f"\nLa correlación entre experiencia e ingresos es: {correlacion.iloc[0,1]:.3f}")
print("Esto indica una correlación positiva moderada-fuerte entre ambas variables.")

Principales hallazgos:

1. Desigualdades detectadas: Existe una brecha salarial significativa entre áreas, 
   con Dirección teniendo el mayor ingreso ($3600) y Recursos Humanos el menor promedio ($2725).

2. Grupos destacados: Los empleados senior en áreas técnicas (IT) y comerciales (Ventas) 
   presentan los mejores perfiles de compensación y experiencia.

3. Anomalías corregidas: Se identificó y corrigió un valor atípico en edad (150 años) 
   y se imputó un valor faltante en ingresos usando la media del área correspondiente.

4. Recomendaciones para la gerencia:
   - Revisar la política salarial en Recursos Humanos para reducir la brecha
   - Implementar un plan de desarrollo para empleados junior en todas las áreas
   - Considerar ajustes salariales basados en años de experiencia

ANÁLISIS ADICIONAL:
- Correlación Experiencia-Ingresos:
                  años_experiencia  ingresos
años_experiencia          1.000000  0.917616
ingresos                  0.917616  1.000000

La correla